# Análisis de Componentes Principales - Paso a Paso

El ACP consiste esencialmente en identificar patrones y correlaciones entre variables para reducir la dimensionalidad de los datos solo cuando tenga sentido, de modo que retengamos la mayor cantidad de información posible.

Se debe mencionar otra técnica conocida como el Análisis del Discriminante Lineal (ADL), que comporta, como su nombre indica, otra transformación lineal de los datos. Mientras que la ACP trata de buscar las Componentes Principales que maximizan la varianza del dato, el ADL intenta buscar direcciones que maximicen la separación (discriminación) entre las diferentes clases, lo cual puede ser muy útil cuando tengamos que aplicar algoritmos de clasificación de patrones.

Mientras que le ACP ignora que haya columnas dedicadas a la clasificación, el LDA tiende a que todos los elementos que pertenecen a clases diferentes estén lo más separados posible, también con técnicas del álgebra lineal.

Entonces, el ejercicio que realizaremos a continuación consistirá en reducir el espacio `m dimensional` original a un subespacio proyectado, un espacio `e dimensional` para reducir el coste computacional del algoritmo que apliquemos a posteriori.

Implementaremos el análisis a mano, paso a paso, de modo que calcularemos los valores propios y cada uno de los vectores propios asociados a cada valor propio, que podrá ser interpretados como la magnitud del vector propio. Si se diera algún vector propio que tuviera una magnitud significativamente grande respecto al resto, lo que ocurrirá es que gran parte de la varianza se la llevará dicho valor, mientras que el resto retendrá menos información. De esta forma podremos ir determinando correctamente la separación.

Habrá que tener en cuenta que los valores del data set se muevan en ordenes de magnitud diferentes.

Teniendo todo ello en cuenta, el procedimiento será el siguiente.

* Normalizar los datos para cada una de las `m` observaciones
* Obtener los vectores y valores propios a partir de la matriz de covarianzas, de correlaciones o con la técnica de singular vector decomposition
* Ordenar los valores propios en orden decreciente y quedarnos con los *p* que se correspondan con los *p* mayores y asi disminuir el número de variables del data set (`p < m`)
* Construir la matriz de proyección `W` a partir de  los `p` vectores propios
* Transformar el dataset original `X` a través de `W` para obtener así datos en el subespacio dimensional de dimensión *p*, que conformará `Y`.

Utilizaremos nuevamente el data set de Iris para ejemplificar el proceso.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../../../GitHub/python-ml-course/datasets/iris/iris.csv")
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


El data set de Iris no se eligió al azar. Este cuenta con una columna de clasificación de la especie. Esta columna deberá permanecer intacta tras la transformación lineal del dataset al aplicar el ACP.

Por tanto, dividiremos el dataset en `X` e `y`, donde `X` estará compuesta por las variables a transformar y en `y` mantendremos las categorías.

In [3]:
X = df.iloc[:,0:4].values
y = df.iloc[:,-1].values

## Representación de datos con Plotly

Al aplicar la transformación a las variables (normalización) dejaremos de tener valores con sentido. Por tanto, antes de ello, realizaremos algunas representaciones de los datos de partida con el paquete `plotly`.

Realizaremos un histograma con todas las variables contenidas dentro del mismo.

In [4]:
import chart_studio.plotly as py
import plotly.tools as tls

from plotly.graph_objs import *

In [5]:
py.sign_in(username='carlosdavila91', api_key='sjP3SuTsus8qjId74CQx')

In [6]:
traces = []
legend = {0:True,1:True,2:True,3:True}

colors = {'setosa': 'rgb(255,127,20)',
         'versicolor': 'rgb(31,220,120)',
         'virginica': 'rgb(44,50,180)'}

for col in range(4):
    for key in colors:
        traces.append(Histogram(x=X[y==key, col], opacity=0.7,
                                xaxis='x%s'%(col+1), marker=Marker(color=colors[key]),
                                name = key, showlegend = legend[col]))
    legend = {0:False,1:False,2:False,3:False}
    
data = Data(traces)
layout = Layout(barmode='overlay',
                xaxis=XAxis(domain=[0,0.25],title="Long. Sépalos (cm)"),
                xaxis2=XAxis(domain=[0.3,0.5], title="Anch. Sépalos (cm)"),
                xaxis3=XAxis(domain=[0.55,0.75], title="Long. Pétalos (cm)"),
                xaxis4=XAxis(domain=[0.8,1.0], title="Anch. Pétalos (cm)"),
                yaxis=YAxis(title="Número de ejemplares"),
                title="Distribución de los rasgos de las diferentes flores Iris")

fig = Figure(data=data, layout=layout)
py.iplot(fig)

/Users/carlosdavila/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:441: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/Users/carlosdavila/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/Users/carlosdavila/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:550: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


/Users/carlosdavila/anac

### Normalización de los datos

Para la normalización de los datos tomaremos el paquete `StandardScaler` de `sklearn`.

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
X_std = StandardScaler().fit_transform(X)

Representamos los datos normalizados para comprobar que las distribuciones permanecen, en este caso, con los valores centrados en 0.

In [9]:
traces = []
legend = {0:True,1:True,2:True,3:True}

colors = {'setosa': 'rgb(255,127,20)',
         'versicolor': 'rgb(31,220,120)',
         'virginica': 'rgb(44,50,180)'}

for col in range(4):
    for key in colors:
        traces.append(Histogram(x=X_std[y==key, col], opacity=0.7,
                                xaxis='x%s'%(col+1), marker=Marker(color=colors[key]),
                                name = key, showlegend = legend[col]))
    legend = {0:False,1:False,2:False,3:False}
    
data = Data(traces)
layout = Layout(barmode='overlay',
                xaxis=XAxis(domain=[0,0.25],title="Long. Sépalos (cm)"),
                xaxis2=XAxis(domain=[0.3,0.5], title="Anch. Sépalos (cm)"),
                xaxis3=XAxis(domain=[0.55,0.75], title="Long. Pétalos (cm)"),
                xaxis4=XAxis(domain=[0.8,1.0], title="Anch. Pétalos (cm)"),
                yaxis=YAxis(title="Número de ejemplares"),
                title="Distribución de los rasgos de las diferentes flores Iris")

fig = Figure(data=data, layout=layout)
py.iplot(fig)

/Users/carlosdavila/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:441: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


/Users/carlosdavila/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


/Users/carlosdavila/anaconda3/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:550: DeprecationWarning:

plotly.graph_objs.XAxis is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.XAxis
  - plotly.graph_objs.layout.scene.XAxis


/Users/carlosdavila/anac

### 1.- Cálculo de la descomposición de valores y vectores propios

#### a) Usando la matriz de covarianzas

Este método es típico en el ámbito de las finanzas.

Recordamos que los valores y vectores propios representan el núcleo fundamental de la ACP. A partir de la matriz de covarianzas o de correlaciones se extraen los vectores propios (componentes principales) que determinan las direcciones del nuevo espacio vectorial, mientras que los valores propios determinan la magnitud. En otras palabras, los valores propios determinan la varianza de los datos a través de los nuevos ejes.

El enfoque más clásico del ACP lo que hace es llevar a cabo la descomposición de la matriz de covarianzas $\Sigma$ de dimensiones `m x m`, donde cada elemento representa la covarianza entre dos de las características que se están analizando.

La covarianza de dos rasgos cualesquiera se puede obtener a través de la siguiente fórmula.

$\sigma_{jk} = \frac{1}{n-1}\sum_{i=1}^m (x_{ij} - \overline{x_j})(x_{ik} - \overline{x_k})$

Y esta fórmula se puede simplificar de la siguiente manera.

$\Sigma = \frac{1}{n-1}((X-\overline{x})^T(X-\overline{x}))$

Donde,

$\overline{x} = \sum_{i=1}^n x_i \in \mathbb R^m$

In [10]:
import numpy as np

In [11]:
mean_vect = np.mean(X_std, axis=0)
mean_vect

array([-4.73695157e-16, -7.81597009e-16, -4.26325641e-16, -4.73695157e-16])

In [12]:
cov_matrix = (X_std - mean_vect).T.dot((X_std-mean_vect))/(X_std.shape[0]-1)
print("La matriz de covarianzas es \n%s"%cov_matrix)

La matriz de covarianzas es 
[[ 1.00671141 -0.11835884  0.87760447  0.82343066]
 [-0.11835884  1.00671141 -0.43131554 -0.36858315]
 [ 0.87760447 -0.43131554  1.00671141  0.96932762]
 [ 0.82343066 -0.36858315  0.96932762  1.00671141]]


También podemos obtener la matriz de covarianzas directamente con `numpy`.

In [13]:
np.cov(X_std.T)

array([[ 1.00671141, -0.11835884,  0.87760447,  0.82343066],
       [-0.11835884,  1.00671141, -0.43131554, -0.36858315],
       [ 0.87760447, -0.43131554,  1.00671141,  0.96932762],
       [ 0.82343066, -0.36858315,  0.96932762,  1.00671141]])

Para resolver el sistema de ecuaciones, como habíamos comentado, habrá que obtener los valores y vectores propios.

In [14]:
eig_vals, eig_vect = np.linalg.eig(cov_matrix)
print("Valores propios \n%s"%eig_vals)
print("Vectores propios \n%s"%eig_vect)

Valores propios 
[2.93808505 0.9201649  0.14774182 0.02085386]
Vectores propios 
[[ 0.52106591 -0.37741762 -0.71956635  0.26128628]
 [-0.26934744 -0.92329566  0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161  0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199  0.63427274  0.52359713]]


Casualmente, los valores propios han aparecido ordenados.

#### b) Usando la matriz de Correlaciones

La matriz de Correlaciones no deja de ser más que la matriz de covarianzas que se obtiene cuando se han normalizado los datos. Por tanto, la descomposición de la matriz de covarianzas después de normalizar los datos es la misma que la descomposición de la matriz de correlaciones sin estandarizar.

In [15]:
corr_matrix = np.corrcoef(X_std.T)
corr_matrix

array([[ 1.        , -0.11756978,  0.87175378,  0.81794113],
       [-0.11756978,  1.        , -0.4284401 , -0.36612593],
       [ 0.87175378, -0.4284401 ,  1.        ,  0.96286543],
       [ 0.81794113, -0.36612593,  0.96286543,  1.        ]])

In [16]:
eig_vals_corr, eig_vect_corr = np.linalg.eig(corr_matrix)
print("Valores propios \n%s"%eig_vals_corr)
print("Vectores propios \n%s"%eig_vect_corr)

Valores propios 
[2.91849782 0.91403047 0.14675688 0.02071484]
Vectores propios 
[[ 0.52106591 -0.37741762 -0.71956635  0.26128628]
 [-0.26934744 -0.92329566  0.24438178 -0.12350962]
 [ 0.5804131  -0.02449161  0.14212637 -0.80144925]
 [ 0.56485654 -0.06694199  0.63427274  0.52359713]]


#### c) Singular Value Decomposition

Esta técnica es la que implementan sistemas sofisticados como las implementaciones en Python o R de la ACP, debido a a su eficacia computacional.

El método nos devuelve los valores `u`, `s` y `v`, donde `u` es la misma matriz de vectores propios que en los casos anteirores. Sin embargo, el equivalente a los valores propios que obtuvimos anteioremente (`s`, en este caso) difiere bastante en rango de valores.

In [17]:
u,s,v = np.linalg.svd(X_std.T)
u

array([[-0.52106591, -0.37741762,  0.71956635,  0.26128628],
       [ 0.26934744, -0.92329566, -0.24438178, -0.12350962],
       [-0.5804131 , -0.02449161, -0.14212637, -0.80144925],
       [-0.56485654, -0.06694199, -0.63427274,  0.52359713]])

In [18]:
s

array([20.92306556, 11.7091661 ,  4.69185798,  1.76273239])

In [19]:
v

array([[ 1.08239531e-01,  9.94577561e-02,  1.12996303e-01, ...,
        -7.27030413e-02, -6.56112167e-02, -4.59137323e-02],
       [-4.09957970e-02,  5.75731483e-02,  2.92000319e-02, ...,
        -2.29793601e-02, -8.63643414e-02,  2.07800179e-03],
       [ 2.72186462e-02,  5.00034005e-02, -9.42089147e-03, ...,
        -3.84023516e-02, -1.98939364e-01, -1.12588405e-01],
       ...,
       [ 5.43380310e-02,  5.12936114e-03,  2.75184277e-02, ...,
         9.89532683e-01, -1.41206665e-02, -8.30595907e-04],
       [ 1.96438400e-03,  8.48544595e-02,  1.78604309e-01, ...,
        -1.25488246e-02,  9.52049996e-01, -2.19201906e-02],
       [ 2.46978090e-03,  5.83496936e-03,  1.49419118e-01, ...,
        -7.17729676e-04, -2.32048811e-02,  9.77300244e-01]])

### 2 - Las componentes principales

Recordamos que el objetivo de la ACP es la disminución de la dimnesionalidad del espacio vectorial inicial meidante la proyección de los datos en subespacios de dimensión más pequeña desde las direcciones que definen los vectores propios desde los ejes.

Además, los vectores propios deberán tener dimensión 1 para que se trate de una base de un espacio vectorial. Comprobamos esto en primer lugar.

In [20]:
for ev in eig_vect:
    print("La longitud del VP es: ", np.linalg.norm(ev))

La longitud del VP es:  0.9999999999999997
La longitud del VP es:  1.0000000000000002
La longitud del VP es:  1.0
La longitud del VP es:  0.9999999999999997


Con la comprobación podemos decir que lo que tenemos es una base de un espacio vectorial original que nos va a servir para llevar a cabo el cálculo de las componentes principales.

Para construir un espacio vectorial de dimensionalidad inferior vamos a buscar cada uno de los valores propios. Los vectores propios que tengan los valores propios menores de la distribución de los datos serán los que vamos a eliminar. Para ello construiremos un ranking con los valores propios para luego elegir el top de ellos que más nos interese (el que represente la mayor parte de la variabilidad de los datos).

In [21]:
eigen_pairs = [(np.abs(eig_vals[i]), eig_vect[:,i]) for i in range(len(eig_vals))]
eigen_pairs

[(2.938085050199993,
  array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654])),
 (0.9201649041624873,
  array([-0.37741762, -0.92329566, -0.02449161, -0.06694199])),
 (0.1477418210449481,
  array([-0.71956635,  0.24438178,  0.14212637,  0.63427274])),
 (0.020853862176462803,
  array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))]

La lista de valores y vectores propios ya viene ordenada. Si no ocurriera esto, el método para ordenar la lista sería el siguiente.

In [22]:
eigen_pairs.sort()
eigen_pairs.reverse()
eigen_pairs

[(2.938085050199993,
  array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654])),
 (0.9201649041624873,
  array([-0.37741762, -0.92329566, -0.02449161, -0.06694199])),
 (0.1477418210449481,
  array([-0.71956635,  0.24438178,  0.14212637,  0.63427274])),
 (0.020853862176462803,
  array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))]

In [23]:
print("Valores propios en orden descendente:")
for ep in eigen_pairs:
    print(ep[0])

Valores propios en orden descendente:
2.938085050199993
0.9201649041624873
0.1477418210449481
0.020853862176462803


Ahora tenemos que determinar cuanta varianza explica cada rasgo. Para ello, utilizaremos la técnica de la varianza explicativa.

Para ello, representaremos un gráfico acumulativo para representar la varianza de cada una de las componentes principales y determinar las que serán necesarias en nuestro dataset final.

In [24]:
total_sum = sum(eig_vals)
var_exp = [(i/total_sum)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

Aprovecharemos la librería `plotly` para hacer la representación combinada de los valores de la varianza explicada en diagrama de barras por un lado, y por el otro, un scatter plot con los puntos unidos por líneas del valor acumulado.

In [25]:
plot1 = Bar(x=["CP %s"%i for i in range(1,5)], y=var_exp, showlegend=False)
plot2 = Scatter(x=["CP %s"%i for i in range(1,5)], y=cum_var_exp, showlegend=True, 
                name = "% de Varianza Explicada Acumulada")

data = Data([plot1,plot2])

layout = Layout(xaxis = XAxis(title="Componentes principales"),
               yaxis = YAxis(title="% de Varianza"),
               title = "Porcentaje de variabilidad explicada por cada componente principal")

fig = Figure(data=data, layout=layout)
py.iplot(fig)

En el gráfico podemos observar que tomando las dos primeras componentes podemos explicar un 95,8% de la información, por lo que podríamos eliminar las componentes 3 y 4 sin perder información relevante.

Con ello, pasaremos a la construcción de la matriz de proyección, que será la que nos sirva para transformar el dataset original al nuevo subespacio vectorial de rasgos.

In [26]:
W = np.hstack((eigen_pairs[0][1].reshape(4,1),
               eigen_pairs[1][1].reshape(4,1)))
W

array([[ 0.52106591, -0.37741762],
       [-0.26934744, -0.92329566],
       [ 0.5804131 , -0.02449161],
       [ 0.56485654, -0.06694199]])

### 3- Proyectar las variables en el nuevo subespacio vectorial

La proyección la llevaremos a cabo a través de la siguiente expresión

$Y = X \cdot W, X \in M(\mathbb R)_{150,4}, W \in M(\mathbb R)_{4,2}, Y \in (\mathbb R)_{150,2}$

In [27]:
Y = X_std.dot(W)

In [31]:
results = []

for name in {'setosa','versicolor','virginica'}:
    result = Scatter(x=Y[y==name,0], y=Y[y==name, 1],
                     mode='markers', name=name, 
                     marker=Marker(size=12, line=Line(color='rgba(220,220,220,0.15)',
                                                     width=0.5),
                                  opacity=0.8))
    results.append(result)
    
data=Data(results)
layout=Layout(showlegend=True, scene=Scene(xaxis=XAxis(title="Componenete Principal 1"),
                                          yaxis=YAxis(title="Componente Principal 2")))

fig=Figure(data=data, layout=layout)
py.iplot(fig)